<a href="https://colab.research.google.com/github/gmazzitelli/GSN2/blob/master/MissioniDotazioni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**per poter essere lle il file in questo modo deve essere un google speradsheet**

quindi:
1. esportare i dati delle missioni come file excell
2. copiare ed incollare il file missioni nel tab dell'anno relativo sul google spreedsheet
3. selezionare nell'apertura del file il tab opportuno




In [133]:
import numpy as np
import pandas as pd
key = "1l-nUkzMRyt4gte4nkMcoBBzDZElnOIeBsGA8zIOVP_E" # foglio missioni
key = "1puUfSofNzvDKDYycQBy0NiKtWcMQSW9iwUDp_GtSXBk" # consuntivo 2019 - ricordarsi di salvare il file come google spreedsheet prima di copiare la chiave


def load_from_gspreadsheet(key, sheet_name):
    # google spredsheet TO url 
    # url = "https://docs.google.com/spreadsheet/ccc?key="+key+"&output=csv&sheet={Sheet2}"
    url = 'https://docs.google.com/spreadsheets/d/{key}/gviz/tq?tqx=out:csv&sheet={sheet_name}&headers=1'.format(
        key=key, sheet_name=sheet_name.replace(' ', '%20'))
    print(url)
    return url
def sp2pd(key, sheet_name, verbose=False):
    # spreadsheet TO Panda
    import pandas as pd
    df = pd.read_csv(load_from_gspreadsheet(key, sheet_name))
    # df = df[df.File_Number.isnull() == False]
    df.fillna('', inplace=True)
    if verbose: print ('Variables: ', df.columns.values)
    return df
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
plt.style.use('fast')
plt.rc('font', size = 14, family ='Sans-Serif')

In [134]:
anno = '2021'
df = sp2pd(key='1l-nUkzMRyt4gte4nkMcoBBzDZElnOIeBsGA8zIOVP_E', sheet_name=anno)
DOTAZIONE = sp2pd(key='1l-nUkzMRyt4gte4nkMcoBBzDZElnOIeBsGA8zIOVP_E', sheet_name='DOTAZIONE')
DOTAZIONE['speso']=np.zeros(len(DOTAZIONE))

https://docs.google.com/spreadsheets/d/1l-nUkzMRyt4gte4nkMcoBBzDZElnOIeBsGA8zIOVP_E/gviz/tq?tqx=out:csv&sheet=2021&headers=1
https://docs.google.com/spreadsheets/d/1l-nUkzMRyt4gte4nkMcoBBzDZElnOIeBsGA8zIOVP_E/gviz/tq?tqx=out:csv&sheet=DOTAZIONE&headers=1


In [135]:
def get_experiment_info(DB=DOTAZIONE, anno='2019', experiment='DARKSIDE', user=''):
  try:
    assegnato = DB[(DB.anno.values==int(anno)) & (DB.esperimento==experiment)].importo.values[0]
    speso = df[user].Totale.str.replace('.', '').str.replace(',', '.').astype(float).sum()
    print (("Speso Missioni = {0:.1f}, Assegnato = {1:.1f}, Rimanete = {2:.1f}").format(speso, assegnato, assegnato-speso))
    DB.loc[DB[(DB.anno.values==int(anno)) & (DB.esperimento==experiment)].index, 'speso']=speso
  except:
    print ("No experimenti or user found")
  return DB

In [136]:
assegnazione = df[df.Doc=="Stanziamento iniziale (assegnazioni)"].Totale.str.replace('.', '').str.replace(',', '.').astype(float).sum()
speso = df[(df.Doc!="Stanziamento iniziale (assegnazioni)")].Totale.str.replace('.', '').str.replace(',', '.').astype(float).sum()
print (("Assegnato = {0:.1f}, Speso = {1:.1f}, Disponibile = {2:.1f}").format(assegnazione,speso, assegnazione-speso))
SpesoT=0.

Assegnato = 13500.0, Speso = 0.0, Disponibile = 13500.0


In [137]:
DOTAZIONE[(DOTAZIONE.anno.values==int(anno))]

,anno,esperimento,importo,totale,speso
8,2021,DTZ,10000,,0.0
9,2021,KM3,2500,,0.0
10,2021,DARKSIDE,1000,13500,0.0


In [138]:
df = df.drop(df.index[0])
df.Intestatario.value_counts()

Series([], Name: Intestatario, dtype: int64)

In [139]:
user = (df.Intestatario=="NAPOLITANO TOMMASO") ^ (df.Intestatario=="FRANCESCHI MASSIMO ALBERTO")
DOTAZIONE = get_experiment_info(DB=DOTAZIONE, anno=anno, experiment='DARKSIDE', user=user)
users = user


Speso Missioni = 0.0, Assegnato = 1000.0, Rimanete = 1000.0


In [140]:
user = (df.Intestatario=="MARTINI AGNESE") ^ (df.Intestatario=="Capone Antonio")
DOTAZIONE = get_experiment_info(DB=DOTAZIONE, anno=anno, experiment='KM3', user=user)
users = users ^ user

Speso Missioni = 0.0, Assegnato = 2500.0, Rimanete = 2500.0


In [141]:
user = (df.Intestatario=="RICCI MARCO") ^ (df.Intestatario=="DE PAOLIS ALBERTO")
DOTAZIONE = get_experiment_info(DB=DOTAZIONE, anno=anno, experiment='LIMADOU_CSN2', user=user)
users = users ^ user
# eccezine per Rici 2 esperimenti stessi users
user = df.Intestatario==""
DOTAZIONE = get_experiment_info(DB=DOTAZIONE, anno=anno, experiment='JemEUSO', user=user)


No experimenti or user found
No experimenti or user found


In [142]:
user = np.invert(users) # tutti gli altri
DOTAZIONE = get_experiment_info(DB=DOTAZIONE, anno=anno, experiment='DTZ', user=user)
#df[user]

Speso Missioni = 0.0, Assegnato = 10000.0, Rimanete = 10000.0


In [143]:
DOTAZIONE[(DOTAZIONE.anno.values==int(anno)) ]

,anno,esperimento,importo,totale,speso
8,2021,DTZ,10000,,0.0
9,2021,KM3,2500,,0.0
10,2021,DARKSIDE,1000,13500,0.0


In [144]:
assegnazione = DOTAZIONE[(DOTAZIONE.anno.values==int(anno)) ].importo.sum()
speso = DOTAZIONE[(DOTAZIONE.anno.values==int(anno)) ].speso.sum()
print (("Assegnato = {0:.1f}, Speso = {1:.1f}, Disponibile = {2:.1f}").format(assegnazione,speso, assegnazione-speso))

Assegnato = 13500.0, Speso = 0.0, Disponibile = 13500.0
